In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
train_full = pd.read_csv('/content/drive/MyDrive/Energy Predictor/train_new.csv').dropna()
test_full = pd.read_csv('/content/drive/MyDrive/Energy Predictor/test_new.csv')

In [3]:
X_train = train_full.drop(['meter_reading'], axis=1)
y_train = train_full['meter_reading']
X_test = test_full.drop(['row_id'], axis=1)

In [4]:
for i in X_test.columns[X_test.isnull().any(axis=0)]:
    X_test[i].fillna(X_test[i].mean(),inplace=True)

In [5]:
# regressor = RandomForestRegressor(max_depth=2, random_state=0)
# regressor.fit(X_train, y_train)
# y_pred = regressor.predict(X_test)

RandomForestRegressor(max_depth=2, random_state=0)

In [5]:
bag_reg = BaggingRegressor(
    DecisionTreeRegressor(random_state=0), n_estimators=5
)
bag_reg.fit(X_train, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(random_state=0),
                 n_estimators=5)

In [6]:
y_pred = bag_reg.predict(X_test)

In [7]:
submit_test = test_full[['row_id','meter']]
submit_test['meter_reading']=pd.Series(y_pred)
submit_test = submit_test.drop(['meter'], axis=1)
submit_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 2 columns):
 #   Column         Dtype  
---  ------         -----  
 0   row_id         int64  
 1   meter_reading  float64
dtypes: float64(1), int64(1)
memory usage: 636.3 MB


<ipython-input-7-a13f4de2402a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit_test['meter_reading']=pd.Series(y_pred)


In [8]:
submit_test.to_csv('/content/drive/MyDrive/Energy Predictor/result_bag.csv', index=False)